# Imports

In [1]:
import pandas as pd
import sqlite3

# Data Loading

In [4]:
conn = sqlite3.connect("../raw_data/ellipsys_test_db.sqlite")
query = "SELECT * FROM oa_trf_src"

df = pd.read_sql(query, conn, index_col='id')

## 

In [5]:
df.head()

,trf,tgtTb,tgtLab,srcTb,srcLab,impact
id,,,,,,
bicprd.fnr_cmn_j,bicprd.fnr_cmn_j.set_valcode_frs_j,vld.tfnr974,rf_cle_cod,dual.sfnr974,nextval,1
bicprd.set_valcode_frs,bicprd.set_valcode_frs.set_valcode_frs,vld.tfnr974,rf_cle_cod,dual.sfnr974,nextval,1
bicprd.spik_lstvaleurs,bicprd.spik_lstvaleurs.updlstval_tpfm7a,bicprd.lstvaleurs,owner,ods.tcifp47,cd_typ_cha,-1
bicprd.spik_lstvaleurs,bicprd.spik_lstvaleurs.updlstval_tpfm7a,bicprd.lstvaleurs,tbl,bicprd.lstvaleurs,valeur,-1
bicprd.spik_lstvaleurs,bicprd.spik_lstvaleurs.updlstval_tpfm7a,bicprd.lstvaleurs,bo_saisie_auto,ods.tcif432,cd_typ_cha,-1


In [6]:
df.shape

(500000, 6)

In [8]:
df = df.astype({"impact": int})

In [10]:
df.dtypes

trf       object
tgtTb     object
tgtLab    object
srcTb     object
srcLab    object
impact     int64
dtype: object

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500000 entries, bicprd.fnr_cmn_j to spiprocs.frm2alim_dm
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   trf     500000 non-null  object
 1   tgtTb   500000 non-null  object
 2   tgtLab  500000 non-null  object
 3   srcTb   500000 non-null  object
 4   srcLab  500000 non-null  object
 5   impact  500000 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 26.7+ MB


In [14]:
df.nunique()

trf        421
tgtTb      366
tgtLab    4054
srcTb      634
srcLab    4138
impact       2
dtype: int64